In [5]:
import pandas as pd
from kiwipiepy import Kiwi

# 1. CSV 불러오기
phishing_df = pd.read_csv("C:/Users/user/Downloads/0708/woogawooga_project/dataset_create/Chaeyeon/피싱데이터에서llm테스트뺀것.csv")
normal_df = pd.read_csv("C:/Users/user/Downloads/0708/woogawooga_project/dataset_create/Chaeyeon/일반통화_남은셋.csv")

# 2. 라벨 추가 및 병합
phishing_df['label'] = '1'
normal_df['label'] = '0'
df_all = pd.concat([phishing_df, normal_df], ignore_index=True)

# 3. Kiwi 초기화
kiwi = Kiwi()

# 4. 추출할 품사 (표제어 기반)
target_pos = ['NNG', 'NNP', 'VV', 'VA']  # 명사, 동사, 형용사

# 5. 표제어 추출 함수
def extract_lemmas(text):
    results = kiwi.analyze(text)
    lemmas = []
    for morph in results[0][0]:  # 첫 번째 해석 결과만 사용
        if morph.tag in target_pos:
            lemmas.append(morph.lemma)  # 표제어
    return ' '.join(lemmas)

# 6. tokenized_text 컬럼 생성 (표제어 기반)
df_all['tokenized_text'] = df_all['text'].astype(str).apply(extract_lemmas)

# 7. 저장
df_all.to_csv("형태소분석_표제어_결과.csv", index=False, encoding='utf-8-sig')
print("표제어 기반 형태소 분석 완료: '형태소분석_표제어_결과.csv' 저장됨")



표제어 기반 형태소 분석 완료: '형태소분석_표제어_결과.csv' 저장됨


In [6]:
df_all['tokenized_text']

0        장판 문의 드리다 전화 드리다 경주 교도소 총무과 하다 되다 재소자 방 보수 공사 ...
1                             교도관 보호 조끼 있다 거래처 있다 원 단가 올리다
2                                  사장 주문 넣다 되다 사장 가다 결제 하다
3                  서울지검 수사관 다르다 앞 명의 도용 사건 관련 확인 차다 연락 드리다
4                         주위 잡음 섞이다 목소리 섞이다 대화 하다 없다 공간 전화
                               ...                        
65892    감사 좋다 하루 되다 문의 사항 없다 기다리다 감사 그러다 기다리다 기다리다 죄송하...
65893    감사 선생 맞다 아이 아이 시간 없다 취소 사유 말씀 부탁 드리다 확인 감사 기다리...
65894                 교재 환불 관련 문의 드리다 확인 감사 환불 사유 되다 감사 감사
65895    감사 감사 기간 지나다 지나다 보내다 맞다 환불 신청 하다 환불 등록 되다 같다 등...
65896    회원 그러다 카드 취소 진행 되다 전체 취소 가능 가능 결제 교재 캐시 넣다 괜찮다...
Name: tokenized_text, Length: 65897, dtype: object

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. 형태소+표제어 처리된 CSV 불러오기
df = pd.read_csv("C:/Users/user/Downloads/0708/woogawooga_project/dataset_create/Chaeyeon/형태소분석_표제어_결과.csv")

# 2. file_name 필터링 (phishing_1281 ~ phishing_2046)
def is_in_range(name):
    if name.startswith("phishing_"):
        try:
            num = int(name.split("_")[1])
            return 1281 <= num <= 2046
        except:
            return False
    return False

df_filtered = df[df["file_name"].apply(is_in_range)]

# 3. tokenized_text 컬럼 NaN 제거 및 문자열 변환
df_filtered["tokenized_text"] = df_filtered["tokenized_text"].astype(str).fillna("")

# 4. file_name 기준으로 대화 단위 결합
grouped = df_filtered.groupby("file_name")["tokenized_text"].apply(lambda x: ' '.join(str(i) for i in x if pd.notnull(i))).reset_index()

# (선택) 라벨도 붙이기
labels = df_filtered[['file_name', 'label']].drop_duplicates()
grouped = pd.merge(grouped, labels, on='file_name')

# 5. TF-IDF 계산 (대화 단위)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(grouped['tokenized_text'])

# 6. 결과 DataFrame으로 변환
words = vectorizer.get_feature_names_out()
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=words)
df_tfidf['file_name'] = grouped['file_name']
df_tfidf['label'] = grouped['label']

# 7. CSV 저장
df_tfidf.to_csv("대화단위_TFIDF_결과.csv", index=False, encoding='utf-8-sig')

print("대화 단위 TF-IDF 계산 완료: '대화단위_TFIDF_결과.csv' 저장됨")


C:\Users\user\AppData\Local\Temp\ipykernel_1260\2427227206.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/user/Downloads/0708/woogawooga_project/dataset_create/Chaeyeon/형태소분석_표제어_결과.csv")
C:\Users\user\AppData\Local\Temp\ipykernel_1260\2427227206.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["tokenized_text"] = df_filtered["tokenized_text"].astype(str).fillna("")


대화 단위 TF-IDF 계산 완료: '대화단위_TFIDF_결과.csv' 저장됨
